In [180]:
import anndata as ad
import pandas as pd
import numpy as np
import scanpy as sc
#import decoupler as dc
import os
import statistics
import seaborn as sns
from scipy.stats import wilcoxon
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage
import matplotlib.pyplot as plt
import pickle

In [181]:
def validate_input_arguments (arguments_list):
    if arguments_list["out_path"] is None:
        print("Please provide an output path")
    elif arguments_list["out_path"][-1] != "/":
        arguments_list["out_path"] = arguments_list["out_path"] + "/"

    if arguments_list["celltype"] is None:
        print("Please provide the name of the metadata field containing cell type annotations")

    if arguments_list["condition"] is None:
        print("Please provide the name of the metadata field containing condition annotations")

    if arguments_list["organism"] is None:
        arguments_list["organism"] = "human"

    if arguments_list["comparison_list"] is None:
        arguments_list["comparison_list"] = np.nan

    if arguments_list["logfc"] is None:
        arguments_list["logfc"] = 0.0

    if arguments_list ["pval"] is None:
        arguments_list["pval"] = 0.05

    if arguments_list ["num_cell_filter"] is None:
        arguments_list["num_cell_filter"] = 0

    if arguments_list["reg"] is None:
        arguments_list["reg"] = load_dorothea_regulon(arguments_list["organism"])

    elif isinstance(arguments_list["reg"], str):
        arguments_list["reg"] = pd.read_csv(arguments_list["reg"], index_col=0)
        arguments_list["reg"] = pd.DataFrame.rename(arguments_list["reg"], columns={"source" : "tf"})

    #is the naming of tf fine like this?
    if not "tf" in arguments_list["reg"] and "target" in arguments_list["reg"] and "weight" in arguments_list["reg"]:
        raise NameException("Not all necessary columns found in regulon table! Please make sure that the regulon has the columns 'source', 'target' and 'weight'!")
    
    if arguments_list["plot"] is None:
        arguments_list["plot"] = True
    elif not isinstance(arguments_list["plot"], (bool)):
        raise ValueError("lot argument must be a boolean value.")
        
   
    return(arguments_list)



In [182]:
class TFObj:
    def __init__(self,
    tf_activities_condition : list,
    tf_activities_cluster : list,
    average_gene_expression : list,
    regulon : pd.DataFrame,
    CTR_input_condition : list,
    CTR_input_cluster : list,
    intracellular_network_condition : list,
    intracellular_network_cluster : list):

        self.tf_activities_condition = tf_activities_condition
        self.tf_activities_cluster = tf_activities_cluster
        self.average_gene_expression = average_gene_expression
        self.regulon = regulon
        self.CTR_input_condition = CTR_input_condition
        self.CTR_input_cluster = CTR_input_cluster
        self.intracellular_network_condition = intracellular_network_condition
        self.intracellular_network_cluster = intracellular_network_cluster

def make_TFOBj(tf_activities_condition : list,
    tf_activities_cluster : list,
    average_gene_expression : list,
    regulon : pd.DataFrame,
    CTR_input_condition : list,
    CTR_input_cluster : list,
    intracellular_network_condition : list,
    intracellular_network_cluster : list):

        tf = TFObj(tf_activities_condition,
            tf_activities_cluster,
            average_gene_expression,
            regulon,
            CTR_input_condition,
            CTR_input_cluster,
            intracellular_network_condition,
            intracellular_network_cluster)

        return tf

In [183]:
def AverageExpression(sub_object, celltype = None, name_iterable = None, outpath = None):
    gene_ids = sub_object.var.index.values
    #cluster line even necessary if not returned?
    #clusters = anndataobject.obs[celltype].cat.categories
    obs = sub_object[:,gene_ids].X.toarray()
    obs = np.expm1(obs)
    avg_df = pd.DataFrame(obs,columns=gene_ids,index= sub_object.obs[celltype])
    avg_df = avg_df.groupby(level=0, observed=False).mean()
    #avg_df.T.to_csv(outpath + name_iterable + "_average_gene_expression_by_cluster_exp.csv")

    return avg_df.T


In [184]:
def eval_pval(p_val):
    p_val = float(p_val)
    if p_val < 0.001: 
      txt = "***"
    elif p_val < 0.01: 
      txt = "**"
    elif p_val < 0.05: 
      txt = "*"
    else:
      txt = "ns"
    return(txt)


def eval_log_fc_tag(log_fc):
    if log_fc >= 1.0: 
      txt = "***"
    elif log_fc > 0.5: 
      txt = "**"
    elif log_fc > 0.0: 
      txt = "*"
    else:
      txt = "ns"
    return(txt)

In [185]:
def create_unfiltered_tf_scores(tf_scores_df, condition, celltype, out_path):   
    summarized_tf_scores_df = tf_scores_df.groupby(celltype, observed = True).mean().T
    #tf_scores_df.groupby(celltype, observed = True).apply(display)
    #agg(["mean", "var"])
    summarized_tf_scores_df.to_csv(out_path + "/unfiltered_tf_scores_" + condition + ".csv")
    return summarized_tf_scores_df

#np.var() returns a value that is different from R's var(), whereas statistics.variance() is the same as R's var()
def save_variable_tf_score(filtered_summarized_tf_scores_df, condition, out_path, plot):
    filtered_summarized_tf_scores_df["var"] = filtered_summarized_tf_scores_df.apply(statistics.variance, axis=1).unique()
    filtered_summarized_tf_scores_df.to_csv(out_path + "/variable_tf_scores_" + condition + ".csv")

    if plot:
        top_variable_tfs = filtered_summarized_tf_scores_df.sort_values("var", ascending=False).head(n=20).drop(columns="var")
        fig, ax = plt.subplots(figsize=(8,7))   
        ax = sns.heatmap(top_variable_tfs, cmap="vlag", center=0, vmin=top_variable_tfs.min(axis=None), cbar_kws={"label": "z-score"})
        ax.set(xlabel="Cell Type", ylabel="Transcription Factor")
        ax.get_figure()
        plt.savefig(out_path + "/tf_activity_top20_variable_" + condition + ".pdf")
        plt.close()

    filtered_summarized_tf_scores_df_var = filtered_summarized_tf_scores_df
    return filtered_summarized_tf_scores_df_var
    

In [186]:
def plot_tf_activity(filtered_summarized_tf_scores_df, tag_mapping, condition, out_path):
    filtered_summarized_tf_scores_df = filtered_summarized_tf_scores_df.drop(columns="var")
    sns.clustermap(filtered_summarized_tf_scores_df, cbar_kws={"label": "z-score"}, figsize=(8,7), cmap="vlag", center=0, yticklabels=False)
    plt.savefig(out_path + "/tf_activity_compressed_" + condition + ".pdf")
    plt.close()

    sns.clustermap(filtered_summarized_tf_scores_df, cbar_kws={"label": "z-score"}, figsize=(8,50), cmap="vlag", center=0, annot= tag_mapping, fmt="")
    plt.savefig(out_path + "/tf_activity_" + condition + ".pdf")
    plt.close()


    

In [187]:
def h_clust(data):
            dist_matrix = pdist(data.T)
            linkage_matrix = linkage(dist_matrix, method = "average")   
            return linkage_matrix

In [220]:
def plot_condition_tf_activities(tf_activity_tables, out_path):
       
   for _, df in tf_activity_tables.items():
        name_df = pd.DataFrame(df)
        significant_res = name_df[name_df["tag"] == "***"]
        significant_genes = np.unique(significant_res["tf"])
        name_df = name_df[name_df['tf'].isin(significant_genes)]
        

        tag_mapping = name_df[["tf", "tag", "CellType"]]
        print(tag_mapping)
        tag_mapping = tag_mapping.pivot(index="tf", columns="CellType", values="tag")
    
        name_df_r = name_df[["r", "tf", "CellType"]]
        name_df_cluster = name_df_r.pivot_table(index = "tf", columns = "CellType", values = "r", aggfunc = "mean")
        name_df_cluster.fillna(0, inplace=True) 

        h_clust_matrix = h_clust(name_df_cluster)

        cluster_map = sns.clustermap(name_df_cluster, cbar_kws={"label": "r"}, figsize=(8,40), cmap="vlag", center=0, annot= tag_mapping,
                      col_linkage= h_clust_matrix, fmt="")
        cluster_map.figure.tight_layout(h_pad=1, w_pad=0.02, rect=[0, 0, 1, 1])
        
        plt.savefig(out_path + "/cluster_condition_activity_difference.pdf")
        plt.close()

In [221]:
def plot_condition_tf_activities_compressed(tf_activity_tables, out_path):
 
  
    for _, df in tf_activity_tables.items():
        name_df = pd.DataFrame(df)
        
        significant_res = name_df[name_df["tag"] == "***"]
        significant_genes = np.unique(significant_res["tf"])
        name_df = name_df[name_df['tf'].isin(significant_genes)]
        
        name_df_r = name_df[["r", "tf", "CellType"]]
        name_df_cluster = name_df_r.pivot_table(index = "tf", columns = "CellType", values = "r", aggfunc = "mean")
        name_df_cluster.fillna(0, inplace=True) 

        h_clust_matrix = h_clust(name_df_cluster)
        name_df_cluster.reset_index()
        sns.clustermap(name_df_cluster, cbar_kws={"label": "r"}, figsize=(8,10), cmap="vlag", center=0, annot= None,
                       yticklabels=False, col_linkage= h_clust_matrix, fmt="")
        
        plt.savefig(out_path + "/cluster_condition_activity_difference_compressed.pdf")
        plt.close()

In [190]:
def map_z_value(tf_scores_df, anndataobject_markers):
    genes = []
    clusters = []
    z_scores = []  

    anndataobject_markers = anndataobject_markers.set_index("gene")
    
    for i in range(len(anndataobject_markers.index)):
        a = anndataobject_markers.index[i]
        for j in range(len(tf_scores_df.index)):
            b = tf_scores_df.index[j]
            if a == b:
                c = anndataobject_markers.columns.get_loc("cluster")
                cluster = anndataobject_markers.iloc[i, c]
                gene_rows = tf_scores_df.iloc[j]
                if isinstance(gene_rows, pd.Series):
                    gene_rows = gene_rows.to_frame().T

                for _, gene_row in gene_rows.iterrows():
                     score = gene_row[cluster]
                     genes.append(a)
                     clusters.append(cluster)
                     z_scores.append(score)

    z_score_df = pd.DataFrame({
        'gene': genes,
        'cluster': clusters,
        'z_score': z_scores
    })

    return z_score_df

In [191]:
def get_significant_tfs_single(tf_activities_sub, celltype, condition, out_path, pval, logfc, name, plot):
    
    #does not work if condition is None
    renamed_condition = condition.replace(",", "_")
    name = name.replace(",", "_")

    single_result_path = out_path + renamed_condition 
    if not os.path.isdir(single_result_path):
        os.mkdir(single_result_path)

    tf_activities_scaled = sc.pp.scale(tf_activities_sub, copy= True)

    #tf_activities_sub = sc.pp.normalize_total(tf_activities_scaled, copy=True)
    #tf_activities_sub = sc.pp.log1p(tf_activities_sub, copy=True) 
    # "warning: seems to be already log transformed"

    number_of_clusters = len(tf_activities_sub.obs["new_annotation"].cat.categories) 


    sc.tl.rank_genes_groups(tf_activities_sub, groupby= "new_annotation", reference="rest", method="wilcoxon", key_added="wilcoxon_markers", corr_method= "bonferroni")
    #sc.tl.filter_rank_genes_groups(anndataobject, min_in_group_fraction=0, key="wilcoxon_markers", key_added= "wilcoxon_markers_filtered")
    

    sc.tl.rank_genes_groups(tf_activities_sub, groupby= "new_annotation", reference="rest", method="t-test_overestim_var", key_added="t_test_overestim_var_markers", corr_method= "bonferroni" )
    #sc.tl.filter_rank_genes_groups(anndataobject, min_in_group_fraction=0, key="t_test_overestim_var_markers", key_added="t_test_overestim_filtered")


    result1 = tf_activities_sub.uns['wilcoxon_markers']
    groups = result1['names'].dtype.names
    anndataobject_markers = pd.DataFrame(
    {group + '_' + key[:1]: result1[key][group]
    for group in groups for key in ['names','logfoldchanges','pvals','pvals_adj']})

    result2 = tf_activities_sub.uns['t_test_overestim_var_markers']
    groups = result2['names'].dtype.names
    anndataobject_markers = pd.DataFrame(
    {group + '_' + key[:1]: result2[key][group]
    for group in groups for key in ['names','logfoldchanges','pvals','pvals_adj']})

    anndataobject_markers_wilcoxon = sc.get.rank_genes_groups_df(tf_activities_sub, group = None, log2fc_min=0, key="wilcoxon_markers")
    
    anndataobject_markers_wilcoxon.rename(columns={"names":"gene", "group": "cluster"}, inplace=True)
    
    #documentation says this won't update the main dataframe but will only overwrite a copy in future pandas versions (after v3)
    anndataobject_markers_wilcoxon["tag"] = None
    anndataobject_markers_wilcoxon["log_fc_tag"] = None
    
    for i in range(len(anndataobject_markers_wilcoxon["pvals_adj"])):
         anndataobject_markers_wilcoxon["tag"].iloc[i,] = eval_pval(anndataobject_markers_wilcoxon["pvals_adj"].iloc[i,])
         anndataobject_markers_wilcoxon["log_fc_tag"].iloc[i,] = eval_log_fc_tag(anndataobject_markers_wilcoxon["logfoldchanges"].iloc[i,])
     

    anndataobject_markers_wilcoxon.to_csv(single_result_path + "/" + name + "_specific_markers_wilcoxon_test.csv",index=0)

    anndataobject_markers_t_over = sc.get.rank_genes_groups_df(tf_activities_sub, group = None, log2fc_min=0, key="t_test_overestim_var_markers")
 
    anndataobject_markers_t_over.rename(columns={"names":"gene", "group": "cluster"}, inplace=True)

    anndataobject_markers_t_over["tag"] = None
    anndataobject_markers_t_over["log_fc_tag"] = None
    for i in range(len(anndataobject_markers_t_over["pvals_adj"])):
         anndataobject_markers_t_over["tag"].iloc[i,] = eval_pval(anndataobject_markers_t_over["pvals_adj"].iloc[i,])
         anndataobject_markers_t_over["log_fc_tag"].iloc[i,] = eval_pval(anndataobject_markers_t_over["logfoldchanges"].iloc[i,])

    anndataobject_markers_t_over.to_csv(single_result_path + "/" + name + "_specific_markers_t_test_overestim_test.csv",index=0)

   #tag mapping wilcoxon
    tag_mapping_wilcox = anndataobject_markers_wilcoxon[["gene", "tag", "log_fc_tag", "cluster", "pvals_adj", "logfoldchanges"]]
    tag_mapping_wilcox = tag_mapping_wilcox[(tag_mapping_wilcox["pvals_adj"] < float(pval)) & 
                              ((tag_mapping_wilcox["logfoldchanges"] > float(logfc)) | 
                              (tag_mapping_wilcox["logfoldchanges"] < -float(logfc)))]

    tag_mapping_wilcox = tag_mapping_wilcox.pivot(index="gene", columns="cluster", values="tag")
    clusters = anndataobject_markers_wilcoxon["cluster"].unique()

    for cluster in clusters:
        if cluster not in tag_mapping_wilcox.columns:
            tag_mapping_wilcox[cluster] = np.nan

    tag_mapping_wilcox.fillna("ns", inplace=True)

    tf_activities_scaled.obs_names = tf_activities_scaled.obs[celltype]
    tf_scores_df = tf_activities_scaled.to_df()
    tf_scores_df.columns.name = None
    unfiltered_tf_scores = create_unfiltered_tf_scores(tf_scores_df, condition, celltype, single_result_path)
   
    #Filter to only include tfs that match the tag_mapping/are markers
    col_num = tf_scores_df.columns.isin(tag_mapping_wilcox.index)  
    filtered_tf_scores_df = tf_scores_df.loc[:, col_num] 
    
    filtered_summarized_tf_scores_df = filtered_tf_scores_df.groupby(celltype, observed = False).mean().T
    filtered_summarized_tf_scores_df.index.name = "gene"
    filtered_summarized_tf_scores_df.to_csv(f'{single_result_path}/tf_scores_{condition}.csv')
    tf_scores_variable_table = save_variable_tf_score(filtered_summarized_tf_scores_df, condition, single_result_path, plot)

    if plot:
        plot_tf_activity(filtered_summarized_tf_scores_df, tag_mapping_wilcox, condition, single_result_path)
    
    #filtered_summarized_tf_scores_df.index = re.sub((".,"), "_", filtered_summarized_tf_scores_df.index)
    anndataobject_markers_wilcoxon_merged = anndataobject_markers_wilcoxon.merge(map_z_value(filtered_summarized_tf_scores_df, anndataobject_markers_wilcoxon),  on=['gene', 'cluster'], how='inner')
    anndataobject_markers_wilcoxon_merged = anndataobject_markers_wilcoxon_merged[anndataobject_markers_wilcoxon_merged.tag != "ns"]
    anndataobject_markers_wilcoxon_merged.dropna(inplace=True)
    res_wilcoxon = anndataobject_markers_wilcoxon_merged[["gene","tag", "cluster", "z_score"]]

    #anndataobject_markers_t_over_merged = anndataobject_markers_t_over.merge(map_z_value_filtered(filtered_summarized_tf_scores_df, anndataobject_markers_t_over),  on=['gene', 'cluster'], how='inner')
    #anndataobject_markers_t_over_merged = anndataobject_markers_t_over_merged[anndataobject_markers_t_over_merged.tag != "ns"]
    #anndataobject_markers_t_over_merged.dropna(inplace=True)
    #res_t_test = anndataobject_markers_t_over[["gene","tag", "cluster", "z_score"]]

    res_wilcoxon.to_csv(single_result_path + "/" + name + "_significant_markers_wilcoxon.csv", index=0)
    #res_t_test.to_csv(single_result_path + "/" + name + "_significant_markers_t_test.csv", index=0)

     #//TODO: 
    #delete one variant and put test type as a variable

    res = {}
    res["cluster"] = res_wilcoxon
    return res

In [192]:
#check if it's okay to not pre set the column type as string or float (maybe no problem in python but was problem in r)

def create_empty_CTR_dataframe():
  
  empty_df = pd.DataFrame(
    columns=[
    "source",
    "target",
    "gene_A",
    "gene_B",
    "type_gene_A",
    "type_gene_B",
    "MeanLR"
    ])
  
  return(empty_df)


def add_entry(source, target, gene_A, gene_B, type_gene_A, type_gene_B, MeanLR):
  df = {"source" : source,
      "target" : target,
      "gene_A" : gene_A,
      "gene_B" : gene_B,
      "type_gene_A" : type_gene_A,
      "type_gene_B" : type_gene_B,
      "MeanLR" : MeanLR}

  return df

In [193]:
def generate_CrossTalkeR_input(tf_activities, gene_expression, out_path, regulon = None, organism = "human"):

  if organism == "human":
    ligands = pd.read_csv("ligands_human.csv")
    R2TF = pd.read_csv("rtf_db_human.csv")
  else: 
    ligands = pd.read_csv("lignads_mouse.csv")
    R2TF = pd.read_csv("rtf_db_mouse.csv")

  R2TF = R2TF.set_index("tf")

  sorted_regulon = regulon[["tf", "target"]]
  sorted_regulon = sorted_regulon.set_index("tf")

  output_list = []

  for row in range(len((tf_activities))):
    
    tf_l = []
    r_tf = []

    #if (tf_activities["z_score"].iloc[row] > 0):
    tf = str(tf_activities["gene"].iloc[row])
    if tf in sorted_regulon.index:
      targets = sorted_regulon.loc[tf, "target"]

    if tf in R2TF.index:
        receptors = R2TF.loc[tf, "receptor"]

    else:
        receptors = []

    tf_ligands = np.intersect1d(targets, ligands)

    if organism == "human":
      if len(tf_ligands) > 0:
        for ligand in tf_ligands:
            expressed = False
            if ligand in gene_expression.index:
              ex_value = gene_expression.loc[ligand, tf_activities.iloc[row, 2]]
              if (ex_value != 0):
                expressed = True

            
            if (expressed == True):
              df_list_l = add_entry(source = tf_activities.iloc[row, 2],
                                                      target = tf_activities.iloc[row, 2],
                                                      gene_A =tf_activities.iloc[row, 0],
                                                      gene_B = ligand,
                                                      type_gene_A = "Transcription_Factor",
                                                      type_gene_B= "Ligand",
                                                      MeanLR= tf_activities.iloc[row, 3]
                                                      )
                
              output_list.append(df_list_l)
              

        if (len(receptors) > 0):
          for receptor in receptors:
            df_list_r = add_entry(source = tf_activities.iloc[row, 2],
                                                target = tf_activities.iloc[row, 2],
                                                gene_A= receptor,
                                                gene_B= tf_activities.iloc[row, 0],
                                                type_gene_A= "Receptor",
                                                type_gene_B= "Transcription Factor",
                                                MeanLR= tf_activities.iloc[row, 3]
                                                )

            output_list.append(df_list_r)
          
                
          
    else: 
      if len(tf_ligands) > 0:
        for ligand in tf_ligands:
            expressed = False
            translations = ligand
            if len(translations) > 0:
              for l in translations:
                print(gene_expression.index)
                if l in gene_expression.index:
                  ex_value = gene_expression.loc[l, tf_activities.iloc[row, 2]]
                  if (ex_value != 0):
                    expressed = True
          
                df_list_l = []
            
                if (expressed == True):
                  df_list_l = add_entry(source = tf_activities.iloc[row, 2],
                                                      target = tf_activities.iloc[row, 2],
                                                      gene_A =tf_activities.iloc[row, 0],
                                                      gene_B = ligand,
                                                      type_gene_A = "Transcription_Factor",
                                                      type_gene_B= "Ligand",
                                                      MeanLR= tf_activities.iloc[row, 3]
                                                      )
                
                  output_list.append(df_list_l)
      
              if (len(receptors) > 0):
                for receptor in receptors:
                  df_list_r = [] 
                  df_list_r = add_entry(tf_activities.iloc[row, 2],
                                                tf_activities.iloc[row, 2],
                                                receptor,
                                                tf_activities.iloc[row, 0],
                                                "Receptor",
                                                "Transcription Factor",
                                                tf_activities.iloc[row, 3]
                                                )
                                                
                  output_list.append(df_list_r)
        #tf_l.to_csv(single_result_path + "/" + renamed_condition + "_ctr_test_l.csv", index=0)
        #r_tf.to_csv(single_result_path + "/" + renamed_condition + "_ctr_test_r.csv", index=0)

    #    r_tf["gene_A"] <- gsub("_", "+", r_tf$gene_A, fixed = TRUE)
    #   r_tf["gene_B"] <- gsub("_", "+", r_tf$gene_B, fixed = TRUE)
    #   tf_l["gene_A"] <- gsub("_", "+", tf_l$gene_A, fixed = TRUE)
    #   tf_l["gene_B"] <- gsub("_", "+", tf_l$gene_B, fixed = TRUE
  
  #output_df = pd.concat([output_df, r_tf], ignore_index=True)
  #output_df = pd.concat([output_df, tf_l], ignore_index=True)
  output_df = pd.DataFrame(output_list)
  output_df.to_csv("tf_l_r.csv", index=0)
  print(output_df)

  return output_df

In [194]:
def generate_intracellular_network(tf_activities, gene_expression, outpath, regulon, organism="human"):

    if len(tf_activities.shape) > 0:
        if organism == "human":
            R2TF = pd.read_csv("rtf_db_human.csv").set_index("tf")
        else:
            R2TF = pd.read_csv("rtf_db_mouse.csv").set_index("tf")

    sorted_regulon = regulon[["tf", "target"]].set_index("tf")

    #preextract values
    tf_genes = tf_activities["gene"].values
    tf_celltypes = tf_activities.iloc[:, 2].values
    tf_scores = tf_activities.iloc[:, 3].values

    TFTG_list = []
    RTF_list = []

    for row in range(len(tf_activities)):
        tf = str(tf_genes[row])
        celltype = tf_celltypes[row]
        tf_score = tf_scores[row]

        targets = sorted_regulon.loc[tf, "target"] if tf in sorted_regulon.index else []
        receptors = R2TF.loc[tf, "receptor"] if tf in R2TF.index else []

        if len(targets) > 0 and len(receptors) > 0:
            for target in targets:
                if target in gene_expression.index:
                    ex_value = gene_expression.at[target, celltype]
                    if ex_value != 0:
                        TFTG_list.append({
                            "celltype": celltype,
                            "TF": tf,
                            "Target_Gene": target,
                            "TF_Score": tf_score
                        })

            for receptor in receptors:
                RTF_list.append({
                    "Receptor": receptor,
                    "TF": tf
                })

    TFTG_df = pd.DataFrame(TFTG_list)
    RTF_df = pd.DataFrame(RTF_list)

    recept_regulon = pd.merge(RTF_df, TFTG_df, on="TF")

    return recept_regulon

In [218]:
from scipy.stats import false_discovery_control
from itertools import combinations

def condition_comparison_significant(tf_activities, out_path, celltype, condition, comparison_list):

    vs_df_dic = {}

    if isinstance(comparison_list[0], str):  # Single pair provided as ["PMF2", "control"]
        comparison_list = [comparison_list]
    
    for vs1, vs2 in comparison_list:

        print(f"vs1: {vs1}, vs2: {vs2}") 

        all_tf_list = tf_activities.var_names

        res = pd.DataFrame()
        for i in tf_activities.obs[celltype].unique(): 
            comparison_sub = tf_activities[(tf_activities.obs[celltype] == i) & (tf_activities.obs[condition].isin([vs1, vs2]))]
            if len(pd.unique(comparison_sub.obs[condition])) == 2:
                condition_table = comparison_sub.obs[[condition]].copy()
                condition_table.columns = ["condition"]
                metadata_counts = condition_table.groupby("condition", observed = False).size()
                
                if (metadata_counts[0] + metadata_counts[1]) > 10:
                    g = comparison_sub.obs[condition].astype("category")
                    g = g.cat.set_categories([vs1, vs2])
                    
                    sc.tl.rank_genes_groups(comparison_sub, groupby= condition,
                                         reference="rest", method="wilcoxon", key_added="condition_comp_markers", corr_method= "bonferroni")
                    
                    #sc.pl.rank_genes_groups_heatmap(comparison_sub, show_gene_labels=True, key="condition_comp_markers")

                    res_tmp = sc.get.rank_genes_groups_df(comparison_sub, group = None, log2fc_min=0, key="condition_comp_markers")
                
                    res_tmp["FDR"] = false_discovery_control(res_tmp["pvals"])

                    group1 = comparison_sub.X[g == vs1]
                    group2 = comparison_sub.X[g == vs2]
                        
                    
                    res_tmp["r"] = res_tmp["scores"] / np.sqrt(len(group1) + len(group2))
                    res_tmp["CellType"] = i

                    res = pd.concat([res, res_tmp], ignore_index=True)

        res_df = res.dropna()

        def assign_significance_tag(fdr):
            if fdr < 0.001:
                return "***"
            elif fdr < 0.01:
                return "**"
            elif fdr < 0.05:
                return "*"
            else:
                return "ns"

        res_df["tag"] = res_df["FDR"].apply(assign_significance_tag)
        
        res_df.rename(columns={"names":"tf", "group": "condition"}, inplace=True)
        res_df.to_csv(f"{out_path}/all_tfs_{vs1}_vs_{vs2}.csv", index=False)

        result_name = f"{vs1}_{vs2}"
        vs_df_dic[result_name] = res_df
        
        return vs_df_dic

In [196]:
def get_significant_tfs(tf_activities_sub, condition, out_path, tf_condition_significant, celltype, pval, logfc, plot):
    
     #does not work if condition is None
    renamed_condition = condition.replace(",", "_")
    #name = name.replace(",", "_")

    single_result_path = out_path + renamed_condition 
    if not os.path.isdir(single_result_path):
        os.mkdir(single_result_path)

    tf_activities_scaled = sc.pp.scale(tf_activities_sub, copy= True)

    #tf_activities_sub = sc.pp.normalize_total(tf_activities_scaled, copy=True)
    #tf_activities_sub = sc.pp.log1p(tf_activities_sub, copy=True) 
    # "warning: seems to be already log transformed"

    number_of_clusters = len(tf_activities_sub.obs["new_annotation"].cat.categories) 


    sc.tl.rank_genes_groups(tf_activities_sub, groupby= "new_annotation", reference="rest", method="wilcoxon", key_added="wilcoxon_markers", corr_method= "bonferroni")
    #sc.tl.filter_rank_genes_groups(anndataobject, min_in_group_fraction=0, key="wilcoxon_markers", key_added= "wilcoxon_markers_filtered")
    

    sc.tl.rank_genes_groups(tf_activities_sub, groupby= "new_annotation", reference="rest", method="t-test_overestim_var", key_added="t_test_overestim_var_markers", corr_method= "bonferroni" )
    #sc.tl.filter_rank_genes_groups(anndataobject, min_in_group_fraction=0, key="t_test_overestim_var_markers", key_added="t_test_overestim_filtered")


    result1 = tf_activities_sub.uns['wilcoxon_markers']
    groups = result1['names'].dtype.names
    anndataobject_markers = pd.DataFrame(
    {group + '_' + key[:1]: result1[key][group]
    for group in groups for key in ['names','logfoldchanges','pvals','pvals_adj']})

    result2 = tf_activities_sub.uns['t_test_overestim_var_markers']
    groups = result2['names'].dtype.names
    anndataobject_markers = pd.DataFrame(
    {group + '_' + key[:1]: result2[key][group]
    for group in groups for key in ['names','logfoldchanges','pvals','pvals_adj']})

    anndataobject_markers_wilcoxon = sc.get.rank_genes_groups_df(tf_activities_sub, group = None, log2fc_min=0, key="wilcoxon_markers")
    
    anndataobject_markers_wilcoxon.rename(columns={"names":"gene", "group": "cluster"}, inplace=True)
    
    #documentation says this won't update the main dataframe but will only overwrite a copy in future pandas versions (after v3)
    anndataobject_markers_wilcoxon["tag"] = None
    anndataobject_markers_wilcoxon["log_fc_tag"] = None
    
    for i in range(len(anndataobject_markers_wilcoxon["pvals_adj"])):
         anndataobject_markers_wilcoxon["tag"].iloc[i,] = eval_pval(anndataobject_markers_wilcoxon["pvals_adj"].iloc[i,])
         anndataobject_markers_wilcoxon["log_fc_tag"].iloc[i,] = eval_log_fc_tag(anndataobject_markers_wilcoxon["logfoldchanges"].iloc[i,])
     

    anndataobject_markers_wilcoxon.to_csv(single_result_path + "/" + renamed_condition + "_specific_markers_wilcoxon_test.csv",index=0)

    anndataobject_markers_t_over = sc.get.rank_genes_groups_df(tf_activities_sub, group = None, log2fc_min=0, key="t_test_overestim_var_markers")
 
    anndataobject_markers_t_over.rename(columns={"names":"gene", "group": "cluster"}, inplace=True)

    anndataobject_markers_t_over["tag"] = None
    anndataobject_markers_t_over["log_fc_tag"] = None
    for i in range(len(anndataobject_markers_t_over["pvals_adj"])):
         anndataobject_markers_t_over["tag"].iloc[i,] = eval_pval(anndataobject_markers_t_over["pvals_adj"].iloc[i,])
         anndataobject_markers_t_over["log_fc_tag"].iloc[i,] = eval_pval(anndataobject_markers_t_over["logfoldchanges"].iloc[i,])

    anndataobject_markers_t_over.to_csv(single_result_path + "/" + renamed_condition + "_specific_markers_t_test_overestim_test.csv",index=0)

   #tag mapping wilcoxon
    tag_mapping_wilcox = anndataobject_markers_wilcoxon[["gene", "tag", "log_fc_tag", "cluster", "pvals_adj", "logfoldchanges"]]
    tag_mapping_wilcox = tag_mapping_wilcox[(tag_mapping_wilcox["pvals_adj"] < float(pval)) & 
                              ((tag_mapping_wilcox["logfoldchanges"] > float(logfc)) | 
                              (tag_mapping_wilcox["logfoldchanges"] < -float(logfc)))]

    tag_mapping_wilcox = tag_mapping_wilcox.pivot(index="gene", columns="cluster", values="tag")
    clusters = anndataobject_markers_wilcoxon["cluster"].unique()

    for cluster in clusters:
        if cluster not in tag_mapping_wilcox.columns:
            tag_mapping_wilcox[cluster] = np.nan

    tag_mapping_wilcox.fillna("ns", inplace=True)

    tf_activities_scaled.obs_names = tf_activities_scaled.obs[celltype]
    tf_scores_df = tf_activities_scaled.to_df()
    tf_scores_df.columns.name = None
    unfiltered_tf_scores = create_unfiltered_tf_scores(tf_scores_df, condition, celltype, single_result_path)
   
    #Filter to only include tfs that match the tag_mapping/are markers
    col_num = tf_scores_df.columns.isin(tag_mapping_wilcox.index)  
    filtered_tf_scores_df = tf_scores_df.loc[:, col_num] 
    
    filtered_summarized_tf_scores_df = filtered_tf_scores_df.groupby(celltype, observed = False).mean().T
    filtered_summarized_tf_scores_df.index.name = "gene"
    filtered_summarized_tf_scores_df.to_csv(f'{single_result_path}/tf_scores_{condition}.csv')
    tf_scores_variable_table = save_variable_tf_score(filtered_summarized_tf_scores_df, condition, single_result_path, plot)

    if plot:
        plot_tf_activity(filtered_summarized_tf_scores_df, tag_mapping_wilcox, condition, single_result_path)
    
    #filtered_summarized_tf_scores_df.index = re.sub((".,"), "_", filtered_summarized_tf_scores_df.index)
    
    anndataobject_markers_wilcoxon_merged = anndataobject_markers_wilcoxon.merge(map_z_value(filtered_summarized_tf_scores_df, anndataobject_markers_wilcoxon),  on=['gene', 'cluster'], how='inner')
    anndataobject_markers_wilcoxon_merged = anndataobject_markers_wilcoxon_merged[anndataobject_markers_wilcoxon_merged.tag != "ns"]
    anndataobject_markers_wilcoxon_merged.dropna(inplace=True)
    res_wilcoxon = anndataobject_markers_wilcoxon_merged[["gene","tag", "cluster", "z_score"]]

    #anndataobject_markers_t_over_merged = anndataobject_markers_t_over.merge(map_z_value_filtered(filtered_summarized_tf_scores_df, anndataobject_markers_t_over),  on=['gene', 'cluster'], how='inner')
    #anndataobject_markers_t_over_merged = anndataobject_markers_t_over_merged[anndataobject_markers_t_over_merged.tag != "ns"]
    #anndataobject_markers_t_over_merged.dropna(inplace=True)
    #res_t_test = anndataobject_markers_t_over[["gene","tag", "cluster", "z_score"]]

    res_wilcoxon.to_csv(single_result_path + "/" + renamed_condition + "_significant_markers_wilcoxon.csv", index=0)
    #res_t_test.to_csv(single_result_path + "/" + renamed_condition + "_significant_markers_t_test.csv", index=0)

    res = {}
    res["cluster"] = res_wilcoxon
    
    #tf_condition_significant["gene"] = gsub(".", "-", tf_condition_significant$gene, fixed = TRUE)
    tf_condition_significant = tf_condition_significant.merge(map_z_value(unfiltered_tf_scores, tf_condition_significant), left_on=None, right_on=None, left_index=False, right_index=False)
    tf_condition_significant.dropna(inplace=True)
    
    res["condition"] = tf_condition_significant
    res["condition"].to_csv(f"{single_result_path}/significant_condition_tf_results_{condition} .csv")

    return res

In [211]:
def IntraTalker_analysis(anndataobject, arguments_list = None):
    
    if (isinstance(anndataobject, str)):
        anndataobject = ad.read_h5ad(anndataobject)

    arguments_list = validate_input_arguments(arguments_list)

    if arguments_list["decoupler_matrix_format"] == "R":
        anndataobject = anndataobject.T

    if not os.path.isdir(arguments_list["out_path"]):
        os.mkdir(arguments_list["out_path"])
        tf_path = arguments_list["out_path"] + "TF_results/"
        os.mkdir(tf_path)
    else:
        tf_path = arguments_list["out_path"] + "TF_results/"


    condition = anndataobject.obs[arguments_list["condition"]]

    #checks for tf activity csv, if nothing there, runs decoupler
    if isinstance(arguments_list["tf_activities"], str):
         tf_activities = ad.read_csv(arguments_list["tf_activities"])
         tf_activities.obs = anndataobject.obs.reindex(tf_activities.obs.index)
         tf_activities.obsm = anndataobject.obsm
         tf_activities.uns = anndataobject.uns

    elif arguments_list["tf_activities"] is None:
         raise NameError("Please attach a csv file with the tf activity values. (For further clarification view the 'Decoupler' section of the vignette.)")
        

    #sets the stage for decision if single condition or comparison analysis is done

    if not pd.isnull(all(arguments_list["comparison_list"])):
        if (len(arguments_list["comparison_list"]) > 0) & (len(pd.unique(anndataobject.obs[arguments_list["condition"]])) < 2):
            arguments_list["comparison_list"] = np.nan
            print("Only one condition was found in the data, although a list of comparisons was provided. The analyses are performed only for the present condition!")

    #code for single condition  analysis

    if pd.isnull(all(arguments_list["comparison_list"])):

        result_list = {}
        gene_expression_list = {}
        CTR_cluster_list = {}
        intranet_cluster_list = {}

        #creates loop until after tf activity score
        
        for name_iterable in anndataobject.obs[arguments_list["condition"]].unique():
            sub_object = anndataobject[anndataobject.obs[arguments_list["condition"]] == name_iterable]
            tf_activities_sub = tf_activities[tf_activities.obs[arguments_list["condition"]] == name_iterable]

            sub_object_avg = AverageExpression(sub_object, name_iterable= name_iterable, celltype = arguments_list["celltype"], outpath= arguments_list["out_path"])
            
            #check how sub object is returned by average expression (subobject.T) or rename return?
            tf_activity_scores = get_significant_tfs_single(tf_activities_sub, celltype = arguments_list["celltype"],
                                                            condition = name_iterable, out_path= tf_path, pval = arguments_list["pval"], 
                                                            logfc = arguments_list["logfc"], name = name_iterable, plot = arguments_list["plot"])

            result_list[name_iterable] = tf_activity_scores
            
            gene_expression_list[name_iterable] = sub_object_avg
            
        
    
            CTR_cluster_list[name_iterable] = generate_CrossTalkeR_input(tf_activity_scores["cluster"],
                                                                            gene_expression_list[name_iterable],
                                                                            arguments_list["out_path"],                                             
                                                                            arguments_list["reg"],
                                                                            arguments_list["organism"])
    
              
            intranet_cluster_list[name_iterable] = generate_intracellular_network(tf_activity_scores["cluster"],
                                                                                  gene_expression_list[name_iterable],
                                                                                  arguments_list["out_path"],
                                                                                  arguments_list["reg"],
                                                                                  arguments_list["organism"])
            
            #also return anndata object with avg expression and tf score as uns layer?

        tf = make_TFOBj(
                tf_activities_condition = list(),
                tf_activities_cluster = result_list,
                average_gene_expression = gene_expression_list,
                regulon = arguments_list["reg"],
                CTR_input_condition = list(),
                CTR_input_cluster = CTR_cluster_list,
                intracellular_network_condition = list(),
                intracellular_network_cluster = intranet_cluster_list)

        with open((arguments_list["out_path"] + "tf.pickle"), "wb") as file:
            pickle.dump(tf, file)

        return tf

    else:
        out_path_compared = (tf_path + "compared")
        if not os.path.isdir(out_path_compared):
            os.mkdir(out_path_compared)

        compared_significant_tfs = condition_comparison_significant(tf_activities, out_path_compared, arguments_list["celltype"], 
                                                                    arguments_list["condition"], arguments_list["comparison_list"])
        
        print("compared tfs done")
        
        if arguments_list["plot"] == True:
            plot_condition_tf_activities(compared_significant_tfs, out_path_compared)
            plot_condition_tf_activities_compressed(compared_significant_tfs, out_path_compared)

    
        result_condition_list = {}
        result_cluster_list = {}
        gene_expression_list = {}
        CTR_condition_list = {}
        CTR_cluster_list = {}
        intranet_condition_list = {}
        intranet_cluster_list = {}

        for name_iterable in anndataobject.obs[arguments_list["condition"]].unique():
            sub_object = anndataobject[anndataobject.obs[arguments_list["condition"]] == name_iterable]
            tf_activities_sub = tf_activities[tf_activities.obs[arguments_list["condition"]] == name_iterable]
            

            compared_tfs = pd.DataFrame({"gene" : pd.Series(dtype="str"), "tag" : pd.Series(dtype="str"), "cluster" : pd.Series(dtype="str")})
            
        
            for result_name, df in zip(map(str, range(len(compared_significant_tfs))), compared_significant_tfs):  
                if name_iterable in result_name:
                    tf_condition_significant = compared_significant_tfs[[result_name]]
                    tf_condition_significant = tf_condition_significant[tf_condition_significant["FDR"] < arguments_list["pval"]]
                    tf_condition_significant = tf_condition_significant[(tf_condition_significant["logfoldchanges"] > float(arguments_list["logfc"])) | (tf_condition_significant["logfoldchanges"] < (0 - float(arguments_list["logfc"])))]
                    tf_condition_significant = tf_condition_significant[["tf", "tag", "CellType"]]
                    tf_condition_significant.rename(columns={"tf":"gene", "CellType": "cluster"}, inplace=True)
                    compared_tfs = pd.concat([compared_tfs, tf_condition_significant])

            print(compared_tfs)
            #compared_tfs["gene"] = compared_tfs["gene"].unique()
         
            
            #name <- str_replace_all(name, "[,;.:-]", "_"),

            sub_object_avg = AverageExpression(sub_object, name_iterable= name_iterable, celltype = arguments_list["celltype"], 
                                               outpath= arguments_list["out_path"])
            
            tf_activity_scores = get_significant_tfs(tf_activities_sub,
                                               name_iterable,
                                               tf_path,
                                               compared_tfs,
                                               celltype = arguments_list["celltype"],
                                               pval = arguments_list["pval"],
                                               logfc = arguments_list["logfc"],
                                               plot = arguments_list["plot"])
            
            print("tf_activities done")

            result_condition_list[name_iterable] = tf_activity_scores["condition"]
            result_cluster_list[name_iterable] = tf_activity_scores["cluster"]

            gene_expression_list[name_iterable] = sub_object_avg
            
        
            CTR_condition_list[name_iterable] = generate_CrossTalkeR_input(tf_activity_scores["condition"],
                                                                            gene_expression_list[name_iterable],
                                                                            arguments_list["out_path"],                                             
                                                                            arguments_list["reg"],
                                                                            arguments_list["organism"])
            
            print("CTR input condition done")
    
            CTR_cluster_list[name_iterable] = generate_CrossTalkeR_input(tf_activity_scores["cluster"],
                                                                            gene_expression_list[name_iterable],
                                                                            arguments_list["out_path"],                                             
                                                                            arguments_list["reg"],
                                                                            arguments_list["organism"])
    
            print("CTR input cluster done")

            #intranet_condition_list[name_iterable] = generate_intracellular_network(tf_activity_scores["condition"],
            #                                                                      gene_expression_list[name_iterable],
            #                                                                      arguments_list["out_path"],
            #                                                                      arguments_list["reg"],
            #                                                                      arguments_list["organism"])
    
            print("intranet input condition done")

            #intranet_cluster_list[name_iterable] = generate_intracellular_network(tf_activity_scores["cluster"],
            #                                                                      gene_expression_list[name_iterable],
            #                                                                      arguments_list["out_path"],
            #                                                                      arguments_list["reg"],
            #                                                                      arguments_list["organism"])
            print("intranet input cluster done")

        tf = make_TFOBj(
                tf_activities_condition = result_condition_list,
                tf_activities_cluster = result_cluster_list,
                average_gene_expression = gene_expression_list,
                regulon = arguments_list["reg"],
                CTR_input_condition = CTR_condition_list,
                CTR_input_cluster = CTR_cluster_list,
                intracellular_network_condition = intranet_condition_list,
                intracellular_network_cluster = intranet_cluster_list)

        with open((arguments_list["out_path"] + "tf.pickle"), "wb") as file:
            pickle.dump(tf, file)
        
        return tf
            

In [222]:
result = IntraTalker_analysis(anndataobject= "LR2TF_test_run/anndata_object.h5ad", arguments_list= {"out_path" : "script_test", 
                                                                                                    "celltype" : "new_annotation",
                                                                                                    "condition" : "protocol", 
                                                                                                    "organism" : "human", 
                                                                                                    "comparison_list" : [["PMF,MF2", "control"], ["test1", "test2"]], 
                                                                                                    "logfc" : "0",
                                                                                                    "pval" : None, 
                                                                                                    "num_cell_filter": None,
                                                                                                    "reg" : "filterd_regulon.csv", 
                                                                                                    "tf_activities" : "decoupler_results.csv", 
                                                                                                    "plot" : True,
                                                                                                    "decoupler_matrix_format" : "Python"})


vs1: PMF,MF2, vs2: control
compared tfs done
          tf  tag       CellType
0        REL  ***  Megakaryocyte
1      KLF13  ***  Megakaryocyte
2       IRF9  ***  Megakaryocyte
3       NRF1  ***  Megakaryocyte
6       HIC1   **  Megakaryocyte
...      ...  ...            ...
1419   FOXP1  ***         Neural
1420  NFATC1  ***         Neural
1433    E2F7    *         Neural
1466    ADNP  ***         Neural
1467   FOXF2  ***         Neural

[764 rows x 3 columns]


C:\Users\Larissa\AppData\Local\Temp\ipykernel_21032\783873770.py:25: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if (metadata_counts[0] + metadata_counts[1]) > 10:
d:\studium\LR2TF_HiWi\.venv\Lib\site-packages\scanpy\tools\_rank_genes_groups.py:640: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
d:\studium\LR2TF_HiWi\.venv\Lib\site-packages\scanpy\tools\_rank_genes_groups.py:455: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, "logfoldchanges"] = np.log2(
d:\studium\LR2TF_HiWi\.venv\Lib\site-packages\scanpy\tools\_rank_genes_groups.py:455: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, "logfoldchanges"] = np.log2(
C:\Users\Larissa\AppData\Local\Temp\ipykernel_2103

Empty DataFrame
Columns: [gene, tag, cluster]
Index: []


d:\studium\LR2TF_HiWi\.venv\Lib\site-packages\scanpy\tools\_rank_genes_groups.py:640: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns[key_added] = {}
d:\studium\LR2TF_HiWi\.venv\Lib\site-packages\scanpy\tools\_rank_genes_groups.py:455: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, "logfoldchanges"] = np.log2(
d:\studium\LR2TF_HiWi\.venv\Lib\site-packages\scanpy\tools\_rank_genes_groups.py:455: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, "logfoldchanges"] = np.log2(
d:\studium\LR2TF_HiWi\.venv\Lib\site-packages\scanpy\tools\_rank_genes_groups.py:455: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, "logfoldchanges"] = np.log2(
d:\studium\LR2TF_HiWi\.venv\Lib\site-packages\scanpy\tools\_rank_genes_groups.py:455: RuntimeWarning: invalid value encountered in log2
  self.stats[group_name, "logfoldchanges"] = np.log2(
d:\studium\LR2TF_HiWi\.v

ValueError: You are trying to merge on object and float64 columns for key 'gene'. If you wish to proceed you should use pd.concat

In [115]:
def add_node_type(df):
    
    df['gene_A'] = np.where(df['type_gene_A'] == 'Ligand', df['gene_A'] + '|L', df['gene_A'])
    df['gene_A'] = np.where(df['type_gene_A'] == 'Receptor', df['gene_A'] + '|R', df['gene_A'])
    df['gene_A'] = np.where(df['type_gene_A'] == 'Transcription Factor', df['gene_A'] + '|TF', df['gene_A'])
    df['gene_B'] = np.where(df['type_gene_B'] == 'Ligand', df['gene_B'] + '|L', df['gene_B'])
    df['gene_B'] = np.where(df['type_gene_B'] == 'Receptor', df['gene_B'] + '|R', df['gene_B'])
    df['gene_B'] = np.where(df['type_gene_B'] == 'Transcription Factor', df['gene_B'] + '|TF', df['gene_B'])
    return df


In [116]:
def combine_LR_and_TF(tf_table, LR_prediction, out_path, condition, add_nodetype = False):

  if isinstance(LR_prediction, pd.DataFrame):
    lr_table = LR_prediction
  else: 
    #fix this
    lr_table = read_csv(LR_prediction)
    lr_table.rownames = lr_table.columns["X"]
    lr_table.columns["X"] = None

  intra_connections = pd.DataFrame()
  for celltype in np.unique([lr_table["source"], lr_table["target"]]):
    lr_filtered_ligands = lr_table[lr_table["source"] == celltype]
    lr_filtered_receptors = lr_table[lr_table["target"] == celltype]
    lr_ligands = np.unique(lr_filtered_ligands["gene_A"])
    lr_receptors = np.unique(lr_filtered_receptors["gene_B"])
    
    tf_table_receptors = tf_table[(tf_table["target"] == celltype) & (tf_table["type_gene_A"] == "Receptor")]
    tf_table_ligands = tf_table[(tf_table["source"] == celltype) & (tf_table["type_gene_B"] == "Ligand")]

    tf_receptor_interactions =  tf_table_receptors[tf_table_receptors["gene_A"].isin(lr_receptors)]
    tf_ligand_interactions = tf_table_ligands[tf_table_ligands["gene_B"].isin(lr_ligands)]

    intra_connections = pd.concat([intra_connections, tf_receptor_interactions, tf_ligand_interactions])
  intra_connections["all_pair"] = (intra_connections["source"] + "/" 
                                    + intra_connections["gene_A"] + "/"
                                    + intra_connections["target"] + "/"
                                    + intra_connections["gene_B"])
    
  intra_connections = intra_connections.drop_duplicates(subset=["all_pair"])
  intra_connections.drop(columns=["all_pair"], inplace=True)

  complete_interactions = pd.concat([intra_connections, lr_table])

  if add_nodetype:
    complete_interactions = add_node_type(complete_interactions)
      
  complete_interactions.to_csv((out_path + "CrossTalkeR_input_" + condition + ".csv"), index=0)
  return(complete_interactions)

In [117]:
def combine_LR_and_TF_unfiltered(tf_table, LR_prediction, out_path, condition, add_nodetype = False):

  if isinstance(LR_prediction, pd.DataFrame):
    lr_table = LR_prediction
  else: 
    #fix this
    lr_table = read_csv(LR_prediction)
    lr_table.rownames = lr_table.columns["X"]
    lr_table.columns["X"] = None


  complete_interactions = pd.concat([tf_table, lr_table])

  if add_nodetype:
    complete_interactions = add_node_type(complete_interactions)
      
  complete_interactions.to_csv((out_path + "CrossTalkeR_input_" + condition + ".csv"), index=0)
  return(complete_interactions)

In [118]:
#ask for list with complexes to try?

def combine_LR_and_TF_complexes(tf_table, LR_prediction, out_path, condition, add_nodetype = False):

  if isinstance(LR_prediction, pd.DataFrame):
    lr_table = LR_prediction
  else: 
    #fix this
    lr_table = read_csv(LR_prediction)
    lr_table.rownames = lr_table.columns["X"]
    lr_table.drop(columns=["X"], inplace=True)

  intra_connections = pd.DataFrame()
  for celltype in np.unique([lr_table["source"], lr_table["target"]]):
    lr_filtered_ligands = lr_table[lr_table["source"] == celltype]
    lr_filtered_receptors = lr_table[lr_table["target"] == celltype]
    lr_ligands = np.unique(lr_filtered_ligands["gene_A"])
    lr_receptors = np.unique(lr_filtered_receptors["gene_B"])

    lr_receptors = pd.Series(lr_receptors)
    contains_complex = lr_receptors.str.contains("_", na=False)
    
    R_with_complex = lr_receptors[contains_complex]
    R_without_complex = lr_receptors[(~contains_complex)]
  
    tf_table_receptors = tf_table[(tf_table["target"] == celltype) & (tf_table["type_gene_A"] == "Receptor")]
 
    tf_receptor_interactions =  tf_table_receptors[tf_table_receptors["gene_A"].isin(R_without_complex)]
    
    complex_df = pd.DataFrame()
    if len(R_with_complex) > 0:
      for complex in R_with_complex:
        receptors = complex.split("_")
        R_TF_with_complex = tf_table_receptors[tf_table_receptors["gene_A"].isin(receptors)]
    
        if len(R_TF_with_complex) == 0:
          continue
        
        R_TF_with_complex.drop_duplicates()
        R_TF_with_complex["gene_A"] = complex
        
        complex_df = pd.concat([complex_df, R_TF_with_complex])

      complex_df.drop_duplicates()

    tf_receptor_interactions = pd.concat([tf_receptor_interactions, complex_df])
    print(tf_receptor_interactions)
    
    tf_table_ligands = tf_table[(tf_table["source"] == celltype) & (tf_table["type_gene_B"] == "Ligand")]

    
    tf_ligand_interactions = tf_table_ligands[tf_table_ligands["gene_B"].isin(lr_ligands)]

    intra_connections = pd.concat([intra_connections, tf_receptor_interactions, tf_ligand_interactions])
  intra_connections["all_pair"] = (intra_connections["source"] + "/" 
                                    + intra_connections["gene_A"] + "/"
                                    + intra_connections["target"] + "/"
                                    + intra_connections["gene_B"])
  print(intra_connections)
  intra_connections = intra_connections.drop_duplicates(subset=["all_pair"])
  intra_connections.drop(columns=["all_pair"], inplace=True)

  complete_interactions = pd.concat([intra_connections, lr_table])

  if add_nodetype:
    complete_interactions = add_node_type(complete_interactions)
      
  complete_interactions.to_csv((out_path + "CrossTalkeR_input_" + condition + ".csv"), index=0)
  return(complete_interactions)

In [ ]:
#WINDOWS
table_ctr = pd.read_csv("LR2TF_test_run\\CTR_LR_complex_edited.csv")
table_exp = pd.read_csv("LR2TF_test_run\\EXP_LR.csv")

ctr_input = combine_LR_and_TF_complexes(result.CTR_input_cluster["control"], table_ctr, "script_test/", "control")
exp_input = combine_LR_and_TF(result.CTR_input_cluster["PMF,MF2"], table_exp, "script_test/", "PMF_MF2")

In [ ]:
#LINUX
table_ctr = pd.read_csv("/home/larissa/Documents/LR2TF_HiWi/LR2TF_test_run/CTR_LR.csv")
table_exp = pd.read_csv("/home/larissa/Documents/LR2TF_HiWi/LR2TF_test_run/EXP_LR.csv")

ctr_input = combine_LR_and_TF(result.CTR_input_cluster["control"], table_ctr, "script_test/", "control")
exp_input = combine_LR_and_TF(result.CTR_input_cluster["PMF,MF2"], table_exp, "script_test/", "PMF_MF2")